**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [3]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
import skvideo.io
import cv2
import json


from keras.models import Sequential,model_from_json, Model
from keras.layers import Input, Flatten
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [4]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [5]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)
            
        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

During the training, ```epsilon``` sets the probability of having the agent explore an action instead of exploiting what it has learnt. This is tantamount to the learning rate for optimizers. An important ```epsilon``` can enable the agent to learn the right action faster. But it is also more likely to make it go backward 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [6]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        '''Met un bord de -1 sur la bordure de la matrice avec une epaisseur de 2. 
        On met un padding de 2 car le rat a une visibilité de deux cellules'''
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [7]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

An array of dimension (n,n) is used to represent the ```board``` as a square grid. 
```position``` represents either the board with ```-1```, the rat with ```1``` and all other cells where the rat isn't with ```0```

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [8]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    
    """
    state has a (5,5,2) shape
    """
    def learned_act(self, s):
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [9]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(e)

        # Update stats
        score += win-lose

        #print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
        #      .format(e, epochs, loss, win, lose, win-lose))
        agent.save()
        
        
        
        ####################
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()
skvideo.setFFmpegPath("/usr/local/bin/")
test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 7.0/19.0. Average score (-24.0)
Win/lose count 9.0/15.0. Average score (-18.0)
Win/lose count 7.5/15.0. Average score (-17.0)
Win/lose count 9.0/9.0. Average score (-12.75)
Win/lose count 9.0/22.0. Average score (-15.4)
Win/lose count 13.5/12.0. Average score (-12.333333333333334)
Win/lose count 14.5/25.0. Average score (-13.571428571428571)
Win/lose count 10.5/9.0. Average score (-11.5)
Win/lose count 10.5/15.0. Average score (-11.222222222222221)
Win/lose count 7.0/12.0. Average score (-11.1)
Final score: -11.1


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [11]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > 100:
            del self.memory[0]

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [12]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0 and e >0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [13]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s]))[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            
            input_states[i] = s_

            target_q[i] = self.model.predict(np.array([s_]))[0]
            
            if game_over_:
                ######## FILL IN
                target_q[i, a_] = r_
                
            else:
                ######## FILL IN
                target_q[i, a_] = r_ + self.discount * self.model.predict(n_s_.reshape(1, 5, 5, self.n_state)).max()
        ######## FILL IN

        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        inp = Input(shape=(5, 5, self.n_state))
        x = Flatten()(inp)
        x = Dense(64, activation="relu")(x)
        x = Dense(4, activation="linear")(x)
        model = Model(inputs=inp, outputs=x)
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [14]:
epochs_train=21 # set small when debugging
epochs_test=11 # set small when debugging

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train{}.mp4'.format(epochs_train-1)))













Epoch 000/021 | Loss 0.0131 | Win/lose count 3.5/5.0 (-1.5)
Epoch 001/021 | Loss 0.0864 | Win/lose count 2.0/4.0 (-2.0)
Epoch 002/021 | Loss 0.0007 | Win/lose count 6.5/4.0 (2.5)
Epoch 003/021 | Loss 0.0060 | Win/lose count 5.0/5.0 (0.0)
Epoch 004/021 | Loss 0.0684 | Win/lose count 8.0/4.0 (4.0)
Epoch 005/021 | Loss 0.0005 | Win/lose count 2.5/2.0 (0.5)
Epoch 006/021 | Loss 0.0583 | Win/lose count 5.0/3.0 (2.0)
Epoch 007/021 | Loss 0.0071 | Win/lose count 6.5/5.0 (1.5)
Epoch 008/021 | Loss 0.0005 | Win/lose count 2.5/0 (2.5)
Epoch 009/021 | Loss 0.0029 | Win/lose count 5.0/2.0 (3.0)
Epoch 010/021 | Loss 0.0013 | Win/lose count 1.0/1.0 (0.0)
Epoch 011/021 | Loss 0.0005 | Win/lose count 11.5/3.0 (8.5)
Epoch 012/021 | Loss 0.0086 | Win/lose count 8.0/3.0 (5.0)
Epoch 013/021 | Loss 0.0039 | Win/lose count 5.0/3.0 (2.0)
Epoch 014/021 | Loss 0.0030 | Win/lose count 4.0/3.0 (1.0)
Epoch 015/021 | Loss 0.0021 | Win/lose count 2.0/2.0 (0.0)
Epoch 016/021 | Loss 0.0048 | Win/lose coun

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [16]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        
        inp = Input(shape=(5, 5, self.n_state))
        x = Conv2D(16, (3, 3), activation='relu')(inp)
        x = Conv2D(16, (3, 3), activation='relu')(x)
        x = Flatten()(x)
        x = Dense(4, activation="linear")(x) 

        model = Model(inputs=inp, outputs=x)
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model


In [17]:
epochs_train=21 # set small when debugging
epochs_test=11 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')


Epoch 000/021 | Loss 0.0016 | Win/lose count 2.5/5.0 (-2.5)
Epoch 001/021 | Loss 0.0147 | Win/lose count 6.0/3.0 (3.0)
Epoch 002/021 | Loss 0.0647 | Win/lose count 9.5/4.0 (5.5)
Epoch 003/021 | Loss 0.0039 | Win/lose count 4.5/5.0 (-0.5)
Epoch 004/021 | Loss 0.0624 | Win/lose count 4.0/2.0 (2.0)
Epoch 005/021 | Loss 0.0028 | Win/lose count 3.0/3.0 (0.0)
Epoch 006/021 | Loss 0.0613 | Win/lose count 7.5/2.0 (5.5)
Epoch 007/021 | Loss 0.0079 | Win/lose count 5.5/6.0 (-0.5)
Epoch 008/021 | Loss 0.0019 | Win/lose count 9.5/4.0 (5.5)
Epoch 009/021 | Loss 0.0017 | Win/lose count 10.0/3.0 (7.0)
Epoch 010/021 | Loss 0.0013 | Win/lose count 9.5/2.0 (7.5)
Epoch 011/021 | Loss 0.0012 | Win/lose count 7.0/4.0 (3.0)
Epoch 012/021 | Loss 0.0036 | Win/lose count 4.5/2.0 (2.5)
Epoch 013/021 | Loss 0.0642 | Win/lose count 7.5/4.0 (3.5)
Epoch 014/021 | Loss 0.0756 | Win/lose count 6.5/4.0 (2.5)
Epoch 015/021 | Loss 0.0492 | Win/lose count 7.5/2.0 (5.5)
Epoch 016/021 | Loss 0.0003 | Win/lose count 0.5/2.0

In [18]:
HTML(display_videos('cnn_train{}.mp4'.format(20)))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [19]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')

print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 7.0/2.0. Average score (10.0)
Win/lose count 6.0/1.0. Average score (10.0)
Win/lose count 3.5/3.0. Average score (7.0)
Win/lose count 3.5/6.0. Average score (4.0)
Win/lose count 14.5/2.0. Average score (8.2)
Win/lose count 7.5/1.0. Average score (9.0)
Win/lose count 8.0/1.0. Average score (9.714285714285714)
Win/lose count 5.5/3.0. Average score (9.125)
Win/lose count 12.0/5.0. Average score (9.666666666666666)
Win/lose count 8.0/4.0. Average score (9.5)
Win/lose count 7.0/5.0. Average score (9.0)
Final score: 9.0
Test of the FC
Win/lose count 7.5/7.0. Average score (1.0)
Win/lose count 6.5/2.0. Average score (5.0)
Win/lose count 1.0/1.0. Average score (3.3333333333333335)
Win/lose count 4.0/5.0. Average score (2.0)
Win/lose count 5.0/6.0. Average score (1.2)
Win/lose count 2.5/2.0. Average score (1.1666666666666667)
Win/lose count 2.5/1.0. Average score (1.4285714285714286)
Win/lose count 2.5/3.0. Average score (1.125)
Win/lose count 7.5/2.0. Average sco

In [20]:
env = Environment(grid_size=size, max_time=T,temperature=0.7)
agent_cnn = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')

print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 15.0/3.0. Average score (24.0)
Win/lose count 21.0/3.0. Average score (30.0)
Win/lose count 16.5/6.0. Average score (27.0)
Win/lose count 27.0/4.0. Average score (31.75)
Win/lose count 12.5/0. Average score (30.4)
Win/lose count 17.0/4.0. Average score (29.666666666666668)
Win/lose count 20.0/3.0. Average score (30.285714285714285)
Win/lose count 11.0/3.0. Average score (28.5)
Win/lose count 34.5/0. Average score (33.0)
Win/lose count 18.0/4.0. Average score (32.5)
Win/lose count 7.5/2.0. Average score (30.545454545454547)
Final score: 30.545454545454547
Test of the FC
Win/lose count 5.0/4.0. Average score (2.0)
Win/lose count 6.5/3.0. Average score (4.5)
Win/lose count 8.0/2.0. Average score (7.0)
Win/lose count 10.5/2.0. Average score (9.5)
Win/lose count 12.0/2.0. Average score (11.6)
Win/lose count 11.0/3.0. Average score (12.333333333333334)
Win/lose count 10.0/6.0. Average score (11.714285714285714)
Win/lose count 12.5/3.0. Average score (12.625)
Wi

In [21]:
HTML(display_videos('cnn_test{}.mp4'.format(epochs_test -1)))

In [22]:
HTML(display_videos('fc_test10.mp4'.format(epochs_test -1)))

We get better performance with the CNN-based model.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [23]:
def greedy_epsilon(e, epochs, e_min = 0.05, bend_rate = 0.3):
    if e/epochs <  bend_rate:
        return e_min + (bend_rate - e/epochs) * (1 - e_min)
    else:
        return e_min

def train_explore(agent,env,epoch,prefix='train_explore'):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        
        
        # This is the only part we change from the original train funciton
        eps = greedy_epsilon(e, epoch)
        agent.set_epsilon(eps)

        
        win = 0
        lose = 0
        
        

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0 and e >0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        
        Environment.__init__(self, grid_size=grid_size, max_time=max_time, temperature=temperature)
        self.malus = np.zeros((self.grid_size, self.grid_size))
        self.malus[self.x, self.y] = 0.1

        
    def reset(self):
        state = Environment.reset(self)
        self.malus = np.zeros(self.board.shape)
        self.malus[self.x, self.y] = 0.1
        malus =  self.malus[self.x-2:self.x+3, self.y-2:self.y+3, np.newaxis] # the rat can only see 2 cells away
        state = np.concatenate((malus, state), axis=2) # stack

        return state

    
    def act(self, action):
        state, reward, game_over = Environment.act(self, action)
        reward -= self.malus[self.x, self.y] # penalize if cell has already been visited
        self.malus[self.x, self.y] = 0.1  # cell has been visited
        malus =  self.malus[self.x-2:self.x+3, self.y-2:self.y+3, np.newaxis] # the rat can only see 2 cells away
        state = np.concatenate((malus, state), axis=2) # concat

        return state, reward, game_over

In [24]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore{}.mp4'.format(epochs_train-1)))

Epoch 000/021 | Loss 0.0642 | Win/lose count 10.5/22.200000000000042 (-11.700000000000042)
Epoch 001/021 | Loss 0.0060 | Win/lose count 16.5/23.80000000000004 (-7.30000000000004)
Epoch 002/021 | Loss 0.0024 | Win/lose count 11.5/17.69999999999999 (-6.199999999999989)
Epoch 003/021 | Loss 0.0110 | Win/lose count 8.5/20.10000000000001 (-11.600000000000009)
Epoch 004/021 | Loss 0.0176 | Win/lose count 12.0/18.499999999999982 (-6.499999999999982)
Epoch 005/021 | Loss 0.0048 | Win/lose count 6.0/20.700000000000035 (-14.700000000000035)
Epoch 006/021 | Loss 0.0033 | Win/lose count 6.0/19.90000000000001 (-13.90000000000001)
Epoch 007/021 | Loss 0.0027 | Win/lose count 4.5/18.599999999999998 (-14.099999999999998)
Epoch 008/021 | Loss 0.0047 | Win/lose count 8.5/18.09999999999999 (-9.59999999999999)
Epoch 009/021 | Loss 0.0095 | Win/lose count 13.0/16.99999999999998 (-3.9999999999999787)
Epoch 010/021 | Loss 0.0954 | Win/lose count 17.5/12.399999999999972 (5.100000000000028)
Epoch 011/021 | Los

In [25]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore{}.mp4'.format(epochs_test-1)))

Win/lose count 20.5/14.09999999999997. Average score (12.800000000000061)
Win/lose count 20.0/16.099999999999966. Average score (10.300000000000065)
Win/lose count 19.5/15.299999999999974. Average score (9.666666666666726)
Win/lose count 13.5/17.199999999999967. Average score (5.400000000000061)
Win/lose count 23.5/13.599999999999978. Average score (8.280000000000056)
Win/lose count 19.5/16.699999999999985. Average score (7.833333333333385)
Win/lose count 12.5/15.599999999999971. Average score (5.828571428571481)
Win/lose count 20.0/12.699999999999976. Average score (6.925000000000052)
Win/lose count 15.5/19.200000000000014. Average score (5.333333333333376)
Win/lose count 26.5/17.699999999999974. Average score (6.560000000000043)
Win/lose count 15.0/18.5. Average score (5.327272727272767)
Final score: 5.327272727272767


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***